In [ ]:
%matplotlib inline
import platform
print('the python version is: ' + platform.python_version())
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType, StructType, StructField, StringType

In [ ]:
# create explicit spark context 
spark = SparkSession.builder.appName("PythonKMeansIris").getOrCreate()

Have a look into the input file first manually (use editor or command on shell). Use the 'csv' function of the read class to read the data and complete schema by hand.
Define a VectorAssembler and transform the input DataFrame into an dataset.

In [ ]:
# generate DataFrame instead of RDD, use DataFrameReader class to parse csv input
# have a look to iris.csv first
df=spark.read.csv("data/iris.csv",
                    schema=StructType([StructField("sepal_length",DoubleType(),True),
                                       StructField("sepal_width",DoubleType(),True),
                                       StructField("petal_length",DoubleType(),True),
                                       StructField("petal_width",DoubleType(),True),
                                       StructField("spezies",StringType(),True)]))
#define a corresponding vector assembler
assembler = VectorAssembler(inputCols=["sepal_length",
                                 "sepal_width",
                                 "petal_length",
                                 "petal_width"],
                      outputCol="features")
#transform dataframe
dataset = assembler.transform(df)

In [ ]:
assembler.explainParams()

**Exercise:** Examine the data structure of the DataFrame by utilizing the functions 'first', 'head', 'show' and 'printSchema'. 

In [ ]:
# examine spakr DataFrame
df.first()

In [ ]:
df.head(10)

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
#RDD style
df.take(3)

**Hint** Convert Spark DataFrame to Pandas dataframe object for plotting

In [ ]:
pdf=df.toPandas()
pdf

In [ ]:
import matplotlib.pyplot as plt
# Plot Sepal
plt.subplot(1, 2, 1)
plt.scatter(pdf.sepal_length, pdf.sepal_width)
plt.title('Sepal')

#Exercise (add a second display about Petal dimensions)
plt.subplot(1,2,2)
#plt.scatter(x.Petal_Length, x.Petal_Width, c=colormap[y.Targets], s=40)
plt.scatter(pdf.petal_length, pdf.petal_width)
plt.title("Petal")

**Question** How many cluters would you expect in the data set

**Exercise:** Define a kmeans-Model and fit it on the dataset

In [ ]:
# Trains a k-means model.
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(dataset)

In [ ]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

In [ ]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
   print(center)

**Exercise:** Visualize the cluster centers in the plot.

In [ ]:
sepalx=[centers[0][0], centers[1][0], centers[2][0]]
sepaly=[centers[0][1], centers[1][1], centers[2][1]]
petalx=[centers[0][2], centers[1][2], centers[2][2]]
petaly=[centers[0][3], centers[1][3], centers[2][3]]

In [ ]:
plt.subplot(1, 2, 1)
#convert DF to pandas-DF
plt.scatter(pdf.sepal_length, pdf.sepal_width)
plt.scatter(sepalx, sepaly, color='r')
plt.title('Sepal')

plt.subplot(1,2,2)
#plt.scatter(x.Petal_Length, x.Petal_Width, c=colormap[y.Targets], s=40)
plt.scatter(pdf.petal_length, pdf.petal_width)
plt.scatter(petalx, petaly, color='r')
plt.title("Petal")

In [ ]:
dataset.take(5)

Compare to truth in data set using 'spezies' variable.

In [ ]:
df.printSchema()

**Exercise:** How many 'spezies' are in the data set? Use the 'select' and 'disctict' transformations to find out.

In [ ]:
df1=df.select('spezies').distinct() #take just first occurance

In [ ]:
df1.take(5)

In [ ]:
#replace strings in Spark DataFrame for spezies with integers
from pyspark.sql.functions import *
df2 = df.withColumn('spezies', regexp_replace('spezies', 'Iris-virginica', '0')
                     ).withColumn('spezies', regexp_replace('spezies', 'Iris-setosa', '1')
                                 ).withColumn('spezies', regexp_replace('spezies', 'Iris-versicolor', '2'))
df2.take(4)

In [ ]:
#typesafe - cast the type of species directly
dfConv = df2.select(df2.spezies.cast('int').alias('spezies'))
dfConv.printSchema()
#convert to Pandas
pdfConv = dfConv.toPandas()
pdfConv

In [ ]:
plt.subplot(1, 2, 1)
#convert DF to pandas-DF
import numpy as np
colormap = np.array(['blue', 'lime', 'black'])
plt.scatter(pdf.sepal_length, pdf.sepal_width, c=colormap[pdfConv.spezies], s=40)
plt.scatter(sepalx, sepaly, color='r')
plt.title('Sepal')

plt.subplot(1, 2, 2)
plt.scatter(pdf.petal_length, pdf.petal_width, c=colormap[pdfConv.spezies], s=40)
plt.scatter(petalx, petaly, color='r')
plt.title('Petalal')


In [ ]:
spark.stop()